## Import SSL data daily

### 0. Initialization

In [14]:
import pandas as pd
from smtutil import hiveutil, oracle_to_hive
from datetime import datetime
from cx_Oracle import DatabaseError
import numpy as np
import csv
import pyspark.sql.functions as F 
import time
b2int = F.udf(hiveutil.binary2int)
from glob import glob
from datetime import timedelta
import os
import shutil
import glob
import sys
sc, spark = hiveutil.get_sc_spark()
hiveutil.register_common_udf(spark)

In [39]:
tmp_data_folder_path=os.path.join(os.path.realpath(".."), "data")
# sys.path.append(os.path.join(os.path.realpath(".."), "data"))

In [4]:
retry_flag = True
retry_count = 0
while retry_flag and retry_count < 10:
    try:
        conn = oracle_to_hive.connect_to_Oracle()
        retry_flag = False
    except DatabaseError as e:
        err, = e.args
        print("Oracle-Error-Message:", err.message)
        retry_count = retry_count + 1
        print(f'Re-conenct after 5s, retry time: {retry_count}')
        time.sleep(5)

Connect to Oracle Successfully


### 1. Set Dates to load

In [21]:
# dest_path='/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ'
# table_name = 'PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM'

#### Please change below code if need to manually run the notebook

In [31]:
today = datetime.now().strftime('%Y%m%d')

sdate = '20220901'  # YYYYMMDD, if none set to today 
edate = '20220901' # YYYYMMDD

In [32]:
loaded_dates = []

if os.path.exists(dest_path)==True:
    for date_id in glob(f'{dest_path}/date_id=*'):
        loaded_dates.append(int(date_id[-8:]))
    
    loaded_dates = [x for x in loaded_dates]
    print(loaded_dates[-1], today)
else:
    print('dest_path not exist')

dest_path not exist


In [34]:
# if not loaded_dates and sdate is None:
#     raise Exception("sdate must be set if no previous data")

sdate = datetime.strptime(str(loaded_dates[-1]), '%Y%m%d') if sdate is None else datetime.strptime(str(sdate), '%Y%m%d')
edate = datetime.strptime(str(end_day), '%Y%m%d') if edate is None else datetime.strptime(str(edate), '%Y%m%d')

delta = edate - sdate 
days = []

for i in range(delta.days + 1):
    day = int((sdate + timedelta(days=i)).strftime('%Y%m%d'))
    if day not in set(loaded_dates):
        days.append(day)
print(f"start_date: {sdate}, end_date: {edate}")

start_date: 2022-09-01 00:00:00, end_date: 2022-09-01 00:00:00


In [36]:
exclude_near_dates = [] 
exclude_near_dates.append(int((datetime.now()).strftime("%Y%m%d")))
exclude_near_dates.append(int((datetime.now() - timedelta(days=1)).strftime("%Y%m%d")))
exclude_near_dates.append(int((datetime.now() - timedelta(days=2)).strftime("%Y%m%d")))
days = [x for x in days if x not in exclude_near_dates]

[20220901]

In [37]:
for day in days:
    date_str = datetime.strptime(str(day), '%Y%m%d').strftime('%Y-%m-%d')
    print(day, date_str)

20220901 2022-09-01


### 2. Load data to local by batch

In [56]:
def load_ssl(date_id,subr_num_partition,load_missing_partition):
    print('subr_num_partition =', subr_num_partition)
    print('1. execute query', datetime.now())
    query = """
    select
        SUBR_NUM,
        SSL_CERT_DOMAIN,
        BYTE_COUNT,
        HOUR_BINARY,
        TOTAL_HIT
    from PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM 
    where ACCESS_DATE =  DATE '{date_str}'
    and MOD(SUBR_NUM, 100) = {subr_num_partition}
    """.format(date_str=date_str, subr_num_partition=subr_num_partition)

    print('2. fetchall and to csv', datetime.now())
#     rows = pd.read_sql(query, con=conn)
    pd.read_sql(query, con=conn).to_csv(f'{tmp_data_folder_path}/tmp.csv',index=False)
    
    print('3. load to spark', datetime.now())
#     spark.createDataFrame(rows, ['SUBR_NUM', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'HOUR_BINARY', 'TOTAL_HIT'])\
#         .createOrReplaceTempView('tmp')
    sdf=spark.read.option("header",True).csv(f'{tmp_data_folder_path}/tmp.csv')
    
    print('4. save to hive', datetime.now()) # <1min
    spark.table('tmp').filter(~F.col("subr_num").isNull()) \
                  .select(F.col("SUBR_NUM").cast("bigint").alias("subr_num"), \
                          F.col("ssl_cert_domain"), \
                          F.col("BYTE_COUNT").cast("bigint").alias("byte_count"), \
                          b2int(F.col("HOUR_BINARY")).cast("int").alias("hour_binary"), \
                          F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                          F.lit(day).alias('date_id')) \
        .repartition(1)\
        .write\
        .mode('append')\
        .partitionBy('date_id')\
        .parquet(f'{tmp_data_folder_path}/VW_IDP_SSL_SED_CALL_SUMM')
    os.remove(f'{tmp_data_folder_path}/tmp.csv')
    print('5. Finish',datetime.now())

In [52]:
## Running the load_ssl
for day in days:
    print(day)
    date_str = datetime.strptime(str(day), '%Y%m%d').strftime('%Y-%m-%d')
    for subr_num_partition in np.arange(1):
        load_ssl(date_str,subr_num_partition,load_missing_partition=False)
## Running the load_ssl second time for those missed subr_num_partition

20220901
subr_num_partition = 0
1. execute query 2022-10-10 17:02:22.823183
2. fetchall and to csv 2022-10-10 17:02:22.823302
3. load to spark 2022-10-10 17:04:39.179370
4. save to hive 2022-10-10 17:04:39.815461
5. Finish 2022-10-10 17:04:52.900089


In [57]:
## Running the load_ssl
for day in days:
    print(day)
    date_str = datetime.strptime(str(day), '%Y%m%d').strftime('%Y-%m-%d')
    for subr_num_partition in np.arange(1):
        load_ssl(date_str,subr_num_partition,load_missing_partition=False)
## Running the load_ssl second time for those missed subr_num_partition

20220901
subr_num_partition = 0
1. execute query 2022-10-10 17:11:53.086042
2. fetchall and to csv 2022-10-10 17:11:53.086204
3. load to spark 2022-10-10 17:13:47.416093
4. save to hive 2022-10-10 17:16:29.993431
5. Finish 2022-10-10 17:17:17.231463


In [85]:
for day in days:
    print(day)
    date_str = datetime.strptime(str(day), '%Y%m%d').strftime('%Y-%m-%d')
    for subr_num_partition in np.arange(100):
        print('subr_num_partition =', subr_num_partition)
        print('1. execute query', datetime.now())
        query = """
        select
            SUBR_NUM,
            SSL_CERT_DOMAIN,
            BYTE_COUNT,
            HOUR_BINARY,
            TOTAL_HIT
        from PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM 
        where ACCESS_DATE =  DATE '{date_str}'
        and MOD(SUBR_NUM, 100) = {subr_num_partition}
        """.format(date_str=date_str, subr_num_partition=subr_num_partition)

        print('2. fetchall and to csv', datetime.now())
        rows = pd.read_sql(query, con=conn)
        
        print('3. load to spark', datetime.now())
        spark.createDataFrame(rows, ['SUBR_NUM', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'HOUR_BINARY', 'TOTAL_HIT'])\
            .createOrReplaceTempView('tmp')

        print('4. save to hive', datetime.now()) # <1min
        spark.table('tmp').filter(~F.col("subr_num").isNull()) \
                      .select(F.col("SUBR_NUM").cast("bigint").alias("subr_num"), \
                              F.col("ssl_cert_domain"), \
                              F.col("BYTE_COUNT").cast("bigint").alias("byte_count"), \
                              b2int(F.col("HOUR_BINARY")).cast("int").alias("hour_binary"), \
                              F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                              F.lit(day).alias('date_id')) \
            .repartition(1)\
            .write\
            .mode('append')\
            .partitionBy('date_id')\
            .parquet('/app/ws-jerry/smc_load/temp/VW_IDP_SSL_SED_CALL_SUMM')

    print(datetime.now())

20220901
subr_num_partition = 0
subr_num_partition = 1
subr_num_partition = 2
subr_num_partition = 3
subr_num_partition = 4
subr_num_partition = 5
subr_num_partition = 6
subr_num_partition = 7
subr_num_partition = 8
subr_num_partition = 9
subr_num_partition = 10
subr_num_partition = 11
subr_num_partition = 12
subr_num_partition = 13
subr_num_partition = 14
subr_num_partition = 15
subr_num_partition = 16
subr_num_partition = 17
subr_num_partition = 18
subr_num_partition = 19
subr_num_partition = 20
subr_num_partition = 21
subr_num_partition = 22
subr_num_partition = 23
subr_num_partition = 24
subr_num_partition = 25
subr_num_partition = 26
subr_num_partition = 27
subr_num_partition = 28
subr_num_partition = 29
subr_num_partition = 30
subr_num_partition = 31
subr_num_partition = 32
subr_num_partition = 33
subr_num_partition = 34
subr_num_partition = 35
subr_num_partition = 36
subr_num_partition = 37
subr_num_partition = 38
subr_num_partition = 39
subr_num_partition = 40
subr_num_partitio

In [ ]:
for day in days:
    print(day)
    date_str = datetime.strptime(str(day), '%Y%m%d').strftime('%Y-%m-%d')
    for subr_num_partition in np.arange(100):
        print('subr_num_partition =', subr_num_partition)
        print('1. execute query', datetime.now())
        query = """
        select
            SUBR_NUM,
            SSL_CERT_DOMAIN,
            BYTE_COUNT,
            HOUR_BINARY,
            TOTAL_HIT
        from PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM 
        where ACCESS_DATE =  DATE '{date_str}'
        and MOD(SUBR_NUM, 100) = {subr_num_partition}
        """.format(date_str=date_str, subr_num_partition=subr_num_partition)

        print('2. fetchall and to csv', datetime.now())
        rows = pd.read_sql(query, con=conn)
        
        print('3. load to spark', datetime.now())
        spark.createDataFrame(rows, ['SUBR_NUM', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'HOUR_BINARY', 'TOTAL_HIT'])\
            .createOrReplaceTempView('tmp')

        print('4. save to hive', datetime.now()) # <1min
        spark.table('tmp').filter(~F.col("subr_num").isNull()) \
                      .select(F.col("SUBR_NUM").cast("bigint").alias("subr_num"), \
                              F.col("ssl_cert_domain"), \
                              F.col("BYTE_COUNT").cast("bigint").alias("byte_count"), \
                              b2int(F.col("HOUR_BINARY")).cast("int").alias("hour_binary"), \
                              F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                              F.lit(day).alias('date_id')) \
            .repartition(1)\
            .write\
            .mode('append')\
            .partitionBy('date_id')\
            .parquet('/app/ws-jerry/smc_load/temp/VW_IDP_SSL_SED_CALL_SUMM')

    print(datetime.now())

### 3. Load missing partition by batch & Move Parquet to 01 Server

In [ ]:
def load_ssl(date_id, missing_partition):
    date_str = datetime.strptime(str(date_id), '%Y%m%d').strftime('%Y-%m-%d')
    print(date_id)
    print('subr_num_partition =', missing_partition)
    print('1. execute query', datetime.now())
    query = """
    select
        SUBR_NUM,
        SSL_CERT_DOMAIN,
        BYTE_COUNT,
        HOUR_BINARY,
        TOTAL_HIT
    from PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM 
    where ACCESS_DATE =  DATE '{date_str}'
    and MOD(SUBR_NUM, 100) = {subr_num_partition}
    """.format(date_str=date_str, subr_num_partition=subr_num_partition)

    print('2. fetchall', datetime.now())
    rows = pd.read_sql(query, con=conn)

    print('3. load to spark', datetime.now())
    spark.createDataFrame(rows, ['SUBR_NUM', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'HOUR_BINARY', 'TOTAL_HIT'])\
        .createOrReplaceTempView('tmp')

    print('4. save to hive', datetime.now()) # <1min
    spark.table('tmp').filter(~F.col("subr_num").isNull()) \
                  .select(F.col("SUBR_NUM").cast("bigint").alias("subr_num"), \
                          F.col("ssl_cert_domain"), \
                          F.col("BYTE_COUNT").cast("bigint").alias("byte_count"), \
                          b2int(F.col("HOUR_BINARY")).cast("int").alias("hour_binary"), \
                          F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                          F.lit(date_id).alias('date_id')) \
        .repartition(1)\
        .write\
        .mode('append')\
        .partitionBy('date_id')\
        .parquet('/app/ws-jerry/smc_load/temp/VW_IDP_SSL_SED_CALL_SUMM')
    print(datetime.now())

In [ ]:
missing_date = [int(x[-8:]) for x in glob.glob('/app/ws-jerry/smc_load/temp/VW_IDP_SSL_SED_CALL_SUMM/date_id=*')]
for date_id in missing_date:
    spark.range(100).createOrReplaceTempView('0_to_99')
    spark.read.parquet(f'/app/ws-jerry/smc_load/temp/VW_IDP_SSL_SED_CALL_SUMM/date_id={date_id}').createOrReplaceTempView('tmp')
    spark.sql("""
    select 
        distinct(MOD(SUBR_NUM,100)) as MOD
    from
        tmp
    group by
        MOD(SUBR_NUM,100)
    order by
        MOD
    """).createOrReplaceTempView('tmp')
    missing_partition_list = spark.sql('select id from 0_to_99 where id not in (select mod from tmp)').select("id").rdd.flatMap(lambda x: x).collect()
    
    if len(missing_partition_list) == 0:
        source = f'/app/ws-jerry/smc_load/temp/VW_IDP_SSL_SED_CALL_SUMM/date_id={date_id}'
        destination = f'/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/date_id={date_id}'
        dest = shutil.move(source, destination)
        print(str(date_id)+' transfer finish')
    else:
        for i in range(len(missing_partition_list)):
            load_ssl(date_id, missing_partition_list[i])

In [ ]:
spark.stop()
sc.stop()